<a href="https://colab.research.google.com/github/sarahjsu/TrafficLLM/blob/main/FactionMLChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.1 MB/s eta 0:00:00


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# Used to securely store your API key
from google.colab import userdata

In [4]:
# Google API Key is stored as a secret variable in google colab
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
#Open Images Here

In [7]:
import PIL.Image
!pip install patool
import patoolib
from PIL import Image


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 2.9 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
!pip install timm
import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [9]:
patoolib.extract_archive('/Self-Driving Cars.v2-version-2.yolov5pytorch.zip')

INFO patool: Extracting /Self-Driving Cars.v2-version-2.yolov5pytorch.zip ...
INFO:patool:Extracting /Self-Driving Cars.v2-version-2.yolov5pytorch.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_18yv5x8m -- "/Self-Driving Cars.v2-version-2.yolov5pytorch.zip"
INFO:patool:running /usr/bin/7z x -o./Unpack_18yv5x8m -- "/Self-Driving Cars.v2-version-2.yolov5pytorch.zip"
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /Self-Driving Cars.v2-version-2.yolov5pytorch.zip extracted to `Self-Driving Cars.v2-version-2.yolov5pytorch' (multiple files in root).
INFO:patool:... /Self-Driving Cars.v2-version-2.yolov5pytorch.zip extracted to `Self-Driving Cars.v2-version-2.yolov5pytorch' (multiple files in root).


'Self-Driving Cars.v2-version-2.yolov5pytorch'

Step 1: Building a Dataloader

In [10]:
import os

In [11]:
#Dataloader Class
class TrafficSignDataset(Dataset):
    def __init__(self, data_root):
        #for image in data_dir, set directory with img label pairs
        self.data_img = data_root+'/images'
        self.data_lbls = data_root+'/labels'
        self.images = []
        self.labels = []
        self.files = []
        for filenames in os.listdir(self.data_img):
            im = Image.open(self.data_img+'/'+filenames, mode="r")
            filename_txt = filenames[:-3]
            filename_txt = filename_txt + "txt"
            lbl = open(self.data_lbls+'/'+filename_txt, "r")
            lbl = lbl.read()

            self.images.append(im)
            self.labels.append(lbl)
            self.files.append(filenames[:-4])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return (self.images[idx], self.labels[idx], self.files[idx])

In [12]:
#note: to evaluate this, you may need to change the path to directory below as I have downloaded the
#self driving cars directory into google colab
data_root = '/content/Self-Driving Cars.v2-version-2.yolov5pytorch/train'

dataset = TrafficSignDataset(data_root)

Step 2: Creating an LLM Wrapper

In [13]:
#LLM Wrapper Class

class LLMWrapper():
  def __init__(self, model, dataset):
    self.dataset = dataset
    self.images = dataset.images
    self.lbls = dataset.labels
    self.files = dataset.files
    self.model = model
    self.results = {}

  def use_model(self, idx):
    img = self.images[idx]
    response = self.model.generate_content(img)
    # to_markdown(response.text)
    response = self.model.generate_content(["Create a dictionary mapping these signs to the number of signs in the picture: 'Green Light', 'Red Light', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop'", img], stream=False)
    response.resolve()

    signs = ['Green Light', 'Red Light', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop']
    output_dict = {}
    for sign in signs:
      sign_idx = response.text.find(sign)
      num_sign = response.text[sign_idx+len(sign)+3]
      output_dict[sign] = int(num_sign)

    self.results[self.files[idx]] = output_dict

In [14]:
wrapper = LLMWrapper(genai.GenerativeModel('gemini-1.5-flash'), dataset)

Step 3: Evaluation

In [18]:
eval_root = '/content/Self-Driving Cars.v2-version-2.yolov5pytorch/test' #path to evaluation directory
eval_dataset = TrafficSignDataset(eval_root)

In [19]:
import time

In [20]:
#getting results from the model
eval_model = LLMWrapper(genai.GenerativeModel('gemini-1.5-flash'), eval_dataset)
for idx in range(len(eval_dataset)):
  eval_model.use_model(idx)
  time.sleep(5)

ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 766.15ms


InternalServerError: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [21]:
eval_model.results

{'road661_png.rf.d9a34539bdac9305fd27a74a4f1854e5': {'Green Light': 0,
  'Red Light': 0,
  'Speed Limit 100': 0,
  'Speed Limit 110': 0,
  'Speed Limit 120': 0,
  'Speed Limit 20': 0,
  'Speed Limit 30': 0,
  'Speed Limit 40': 1,
  'Speed Limit 50': 0,
  'Speed Limit 60': 0,
  'Speed Limit 70': 0,
  'Speed Limit 80': 0,
  'Speed Limit 90': 0,
  'Stop': 0},
 'road582_png.rf.969c8df59155ed6b008bf8e0f91a6e1a': {'Green Light': 1,
  'Red Light': 2,
  'Speed Limit 100': 0,
  'Speed Limit 110': 0,
  'Speed Limit 120': 0,
  'Speed Limit 20': 0,
  'Speed Limit 30': 0,
  'Speed Limit 40': 0,
  'Speed Limit 50': 0,
  'Speed Limit 60': 0,
  'Speed Limit 70': 0,
  'Speed Limit 80': 0,
  'Speed Limit 90': 0,
  'Stop': 0},
 'FisheyeCamera_1_00153_png.rf.c6d0f5f472c344bc1dbb288fa6baca1c': {'Green Light': 0,
  'Red Light': 0,
  'Speed Limit 100': 0,
  'Speed Limit 110': 0,
  'Speed Limit 120': 0,
  'Speed Limit 20': 0,
  'Speed Limit 30': 0,
  'Speed Limit 40': 0,
  'Speed Limit 50': 0,
  'Speed Limit 

In [32]:
signs = ['Green Light', 'Red Light', 'Speed Limit 100', 'Speed Limit 110', 'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50', 'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop']

num_errors = 0
errors_list = []

for idx in range(len(eval_dataset)):
  image, label, filename = eval_dataset[idx]

  correct_sign_num = int(label[0])
  correct_sign = signs[correct_sign_num]

  if filename in eval_model.results.keys():
    results_dict = eval_model.results[filename]

    if results_dict[correct_sign] == 0:
      num_errors += 1
      errors_list.append({'filename': filename, 'text output': results_dict})

percent_error = num_errors/len(eval_model.results.keys())


In [34]:
import csv

fields = ['filename', 'text output']

with open('results.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames = fields)

    writer.writeheader()

    writer.writerows(errors_list)

    writer = csv.writer(file)

    writer.writerow(["Percent Error:", 1-percent_error])